In [71]:
map_rels_no = {'COM':0, 'CONTR':1, 'CORR':2, 'QAP':3, 'ACK':4,'ELAB':5,
                 'CLARIFQ':6, 'COND':7, 'CONTIN':8, 'RES':9, 'EXPL':10, 'QELAB':11,
                 'ALT':12, 'NARR':13, 'CONFQ':14, 'SEQ':15, 'NULL':16}

# reverse_relations = {0:'COM', 1: 'CONTR', 2 :'CORR', 3: 'QAP', 4: 'ACK', 5: 'ELAB',
#             6: 'CLARIFQ', 7: 'COND', 8: 'CONTIN', 9: 'RES', 10: 'EXPL',
#             11: 'QELAB', 12: 'ALT', 13: 'NARR', 14: 'CONFQ', 15: 'SEQ', 16: 'NULL'}

reverse_relations = {'Comment':0, 'Contrast':1, 'Correction':2, 'Question-answer_pair':3, 'Acknowledgement':4,'Elaboration':5,
                 'Clarification_question':6, 'Conditional':7, 'Continuation':8, 'Result':9, 'Explanation':10, 'Q-Elab':11,
                 'Alternation':12, 'Narration':13, 'Confirmation_question':14, 'Sequence':15, 'Break':16}

In [73]:
map_rels_str = {'Comment':'COM', 'Contrast':'CONTR', 'Correction':'CORR', 'Question-answer_pair':'QAP', 'Acknowledgement':'ACK','Elaboration':'ELAB',
                 'Clarification_question':'CLARIFQ', 'Conditional':'COND', 'Continuation':'CONTIN', 'Result':'RES', 'Explanation':'EXPL', 'Q-Elab':'QELAB',
                 'Alternation':'ALT', 'Narration':'NARR', 'Background':'BACK', 'Parallel':'PAR', 'Sequence':'SEQ', 'Question_answer_pair':'QAP',  
                 'Q_Elab':'QELAB', 'Confirmation_question' : 'CONFQ'}


## DU and relation counts

In [101]:
#STAC COUNTS real quick
import json
from collections import Counter, defaultdict
home=%pwd
stac_path = home + '/stac_linguistic_flat_train.json'
bad_path = '/home/kate/minecraft_utils/llm_annotator/stac/stac_linguistic_corrected/train_data.json'
squished_path = '/home/kate/minecraft_utils/llm_annotator/stac/stac_squished_corrected/train_data.json'
#msdc_path = '/home/kate/MSDC_linagora/data/TRAIN+VAL_407_bert.json'
msdc_path = '/home/kate/MSDC_linagora/data/TEST_133.json'



In [102]:
with open(bad_path, 'r') as j:
    jfile = json.load(j)
    data = jfile

In [103]:
data[0].keys()

dict_keys(['id', 'edus', 'relations'])

In [104]:
#num edus for stac squished
eeu = 0
edu = 0
for d in data:
    for e in d['edus']:
        if e['speaker'] in ['Server', 'UI']:
            eeu+= 1
        else:
            edu += 1

In [105]:
eeu, edu

(0, 11898)

In [86]:
eeu, edu

(11707, 11566)

In [79]:
data[0]['edus'][:20]

[{'text': "It's inca's turn to roll the dice.", 'speaker': 'Server'},
 {'text': 'inca rolled a 3 and a 3.', 'speaker': 'Server'},
 {'text': 'inca gets 1 wheat. nareik15 gets 1 wheat.', 'speaker': 'Server'},
 {'text': 'inca has 7 resources. nareik15 has 10 resources. yiin has 3 resources. Gaeilgeoir has 4 resources.',
  'speaker': 'UI'},
 {'text': 'anyone got sheep?', 'speaker': 'inca'},
 {'text': 'I can give anything but clay', 'speaker': 'inca'},
 {'text': 'sorry,', 'speaker': 'Gaeilgeoir'},
 {'text': "I don't.", 'speaker': 'Gaeilgeoir'},
 {'text': 'sorry', 'speaker': 'nareik15'},
 {'text': 'inca ended their turn.', 'speaker': 'UI'},
 {'text': "It's nareik15's turn to roll the dice.", 'speaker': 'Server'},
 {'text': 'nareik15 played a Year of Plenty card.', 'speaker': 'Server'},
 {'text': 'nareik15 received 2 clay from the bank.', 'speaker': 'Server'},
 {'text': "It's nareik15's turn to roll the dice.", 'speaker': 'Server'},
 {'text': 'nareik15 rolled a 3 and a 2.', 'speaker': 'Server

In [62]:
data[0]['relations'][:3]

[{'x': 0, 'y': 1, 'type': 'Acknowledgement'},
 {'x': 0, 'y': 2, 'type': 'Continuation'},
 {'x': 2, 'y': 3, 'type': 'Elaboration'}]

In [69]:
mpdu = 0
mpdu_nums = []
for d in data:
    
    rels_count = Counter([g['y'] for g in d['relations']])
    mps = [r[1] for r in rels_count.items() if r[1] > 1] 
    mpdu_nums.extend(mps)
    mpdu += len(mps)

print("mpdus: ", mpdu)

mpdus:  1476


In [70]:
Counter(mpdu_nums)

Counter({2: 1438, 3: 35, 4: 3})

### FLATTEN ALL DATA

#### backwards relations

In [59]:
bkwds = defaultdict(list)
for d in data:
    for rel in d['relations']:
        if rel['y'] < rel['x']:
            bkwds[map_rels_str[rel['type']]].append(rel['y'] - rel['x'])

In [60]:
for k in bkwds.keys():
    print('{} rel : {}'.format(k, len(bkwds[k])))

COM rel : 147
COND rel : 7


STAC SQUISHED
COND rel : 68
COM rel : 57
EXPL rel : 4
BACK rel : 10
ELAB rel : 3

STAC LINGUISTIC
COND rel : 67
CONTIN rel : 1
COM rel : 61
BACK rel : 8
ELAB rel : 3
EXPL rel : 8

In [ ]:
edus = 0
eeus = 0
for d in stac['dialogues']:
    total_dus = len(d['edus'])
    eeu_count = len([e for e in d['edus'] if e['speaker'] in ['UI', 'Server']])
    eeus += eeu_count
    edu_count = total_dus - eeu_count
    edus += edu_count

print("edus: ", edus)
print("eeus: ", eeus)

In [ ]:
cdus = 0
for d in stac['dialogues']:
    cdus += len(d['cdus'])

print("cdus: ", cdus)


In [ ]:
rels = 0
for d in stac['dialogues']:
    rels += len(d['relations'])

print("rels: ", rels)